In [2]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import random
import os


In [7]:
orfs = pd.read_csv("../results/ecoli_orfs_v2.csv")
orfs.head()


,start,end,length,frame,strand,protein
0,107,500,393,2,-,MVFSIIATRWLVILAGAERTVATCPCLALLSRISATRRKRSAFATD...
1,336,2799,2463,0,+,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...
2,2800,3733,933,1,+,MVKVYAPASSANMSVGFDVLGAAVTPVDGALLGDVVTVEAAETFSL...
3,3511,4162,651,0,-,MSHCRSGITGGRGQNGHWLITRNMGQHLCHKATAEIFKCQRWPVEQ...
4,3733,5020,1287,1,+,MKLYNLKDHNEQVSFAQAVTQGLGKNQGLFFPHDLPEFSLTEIDEM...


In [8]:
random.seed(42)
sampled_orfs = orfs.sample(50)


In [9]:
orfs = orfs[orfs["length"] >= 300]


In [10]:
genome = next(SeqIO.parse("../data/genomes/ecoli.fasta", "fasta"))
genome_seq = genome.seq


In [11]:
gene_records = []

for i, row in sampled_orfs.iterrows():
    seq = genome_seq[row["start"]:row["end"]]
    record = SeqIO.SeqRecord(
        seq,
        id=f"gene_{i}",
        description=f"start={row['start']},end={row['end']}"
    )
    gene_records.append(record)

SeqIO.write(gene_records, "../data/genes/ecoli_50_genes.fasta", "fasta")


50

In [12]:
cols = [
    "qseqid","sseqid","pident","length","mismatch","gapopen",
    "qstart","qend","sstart","send","evalue","bitscore"
]

shi = pd.read_csv("../results/ecoli_vs_shigella.tsv", sep="\t", names=cols)


In [13]:
def filter_hits(df):
    return df[
        (df["evalue"] < 1e-10) &
        (df["pident"] >= 70)
    ]

shi_f = filter_hits(shi)


In [14]:
genes_shi = set(shi_f["qseqid"])
genes_all = set(sampled_orfs.index.map(lambda x: f"gene_{x}"))


In [15]:
print(f"Shigella: {len(genes_shi)}")


Shigella: 47
